<a href="https://colab.research.google.com/github/KTDLVB-UD-CQ2018/question_classification/blob/master/training_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [1]:
import pandas as pd 
import re
import string
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline  import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import metrics

import pickle
import joblib

**Check sklearn version**

In [2]:
sklearn_version = sklearn.__version__
print(sklearn_version)

0.22.2.post1


**Get data from github**

In [3]:
!wget https://github.com/KTDLVB-UD-CQ2018/question_classification/blob/master/app/VietnamseQuestionDataset.xlsx?raw=true -O dataset.xlsx

--2021-04-26 15:52:24--  https://github.com/KTDLVB-UD-CQ2018/question_classification/blob/master/app/VietnamseQuestionDataset.xlsx?raw=true
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/KTDLVB-UD-CQ2018/question_classification/raw/master/app/VietnamseQuestionDataset.xlsx [following]
--2021-04-26 15:52:24--  https://github.com/KTDLVB-UD-CQ2018/question_classification/raw/master/app/VietnamseQuestionDataset.xlsx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/KTDLVB-UD-CQ2018/question_classification/master/app/VietnamseQuestionDataset.xlsx [following]
--2021-04-26 15:52:24--  https://raw.githubusercontent.com/KTDLVB-UD-CQ2018/question_classification/master/app/VietnamseQuestionDataset.xlsx
Resolving raw.githubusercontent.com (raw.githubuserconten

In [4]:
df = pd.read_excel("/content/dataset.xlsx", sheet_name = 0, header=0, skiprows = 3, usecols = [1,2,4])

**Làm sạch dữ liệu bằng cách loại bỏ các dấu câu, chữ số, đường link,..**

In [5]:
def Clean_Corpus(sent):
  sent = sent.lower()
  sent = re.sub('\[.*?\]', '', sent)
  sent = re.sub('https?://\S+|www\.\S+', '', sent)
  sent = re.sub('<.*?>+', '', sent)
  sent = re.sub('[%s]' % re.escape(string.punctuation), '', sent)
  sent = re.sub('\n', '', sent)
  sent = re.sub('\w*\d\w*', '', sent)
  return sent


In [6]:
df['Câu hỏi'] = df['Câu hỏi'].apply(Clean_Corpus)

**Fix some issues with incorrect labels**

In [7]:
def normalize_categories(cat):
  res = cat
  if res == "ENT":
    res = "ENTY"
  if res == "Select":
    res = "SELECT"
  if res == "NUM.data":
    res = "NUM.date"
  return res

In [8]:
df['LOẠI'] = df['LOẠI'].apply(normalize_categories)

**Phân ra tập Train và tập Test**

In [9]:
train_df = df[df['DATASET'] == 'TRAIN']
test_df = df[df['DATASET'] == 'TEST']

In [10]:
train_df.shape

(1837, 3)

In [11]:
test_df.shape

(442, 3)

**Lấy ra các dữ liệu cần cho mô hình**


Vì trong dataframe chỉ có 2 cột 'Câu hỏi' và 'LOẠI' là hữu ích cho mô hình

In [12]:
X_train = train_df['Câu hỏi'].to_list()
y_train = train_df['LOẠI'].to_list()
X_test = test_df['Câu hỏi'].to_list()
y_test = test_df['LOẠI'].to_list()

**Phân tập Train ban đầu thành 2 tập Train và Validation**

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.33, random_state = 0, stratify=y_train)

In [14]:
len(X_train)

1230

In [15]:
len(X_val)

607

**Create TF-IDF transformer**

In [16]:
tfidf = TfidfTransformer()

# **Huấn luyện theo mô hình Naive Bayes**

**Thử nghiệm chọn ra mô hình tốt nhất**

In [17]:
li_fit_prior = [True, False]
li_alpha = [0.9,  0.99, 1.0, 1.1, 1.11, 1.111, 1.2, 1.21]
li_ngram = [(1,1), (1,2), (1,3), (2,2), (2,3), (3,3)]

best_alpha = 0 
best_fit_prior = False
best_ngram = (0,0)
best_score = 0.0

for fit_prior in li_fit_prior:
  for alpha in li_alpha:
    for ngram in li_ngram:
      vect = CountVectorizer(ngram_range=ngram)
      pipe = make_pipeline(vect, tfidf)
      pipe.fit_transform(X_train)
      pipe.transform(X_val)
      nb = MultinomialNB(alpha = alpha, fit_prior = fit_prior)
      full_pipeline = make_pipeline(pipe, nb)
      full_pipeline.fit(X_train, y_train)
      y_pred_class = full_pipeline.predict(X_val)
      score = metrics.accuracy_score(y_val, y_pred_class)
      if score > best_score:
        best_alpha = alpha
        best_fit_prior = fit_prior
        best_ngram = ngram
        best_score = score

In [18]:
print(best_alpha)
print(best_fit_prior)
print(best_ngram)
print(best_score)

1.1
False
(1, 3)
0.728171334431631


**Huấn luyện theo mô hình tốt nhất vừa tìm được**


In [19]:
nb = MultinomialNB(alpha = best_alpha, fit_prior = best_fit_prior)
vect = CountVectorizer(ngram_range=best_ngram)
pipe = make_pipeline(vect, tfidf)
pipe.fit_transform(X_train, y_train)
pipe.transform(X_val)
pipe.transform(X_test)
full_pipeline = make_pipeline(pipe, nb)

**Thực hiện dự đoán trên tập Test**

In [20]:
full_pipeline.fit(X_train, y_train)
y_pred_class = full_pipeline.predict(X_test)
print('Accuracy: ')
print(metrics.accuracy_score(y_test, y_pred_class))


Accuracy: 
0.7239819004524887


**Dump models**

In [21]:
joblib.dump(full_pipeline, "pipeline_nb.joblib")

['pipeline_nb.joblib']

# **Huấn luyện theo mô hình Logistic Regression**

**Thử nghiệm chọn ra mô hình tốt nhất**

In [22]:
li_C = [10, 20, 30, 40, 50, 60, 70, 80]
li_ngram = [(1,1), (1,2), (1,3), (2,2), (2,3)]

best_C = 0
best_ngram = (0,0)
best_score = 0.0

for C in li_C:
    for ngram in li_ngram:
      vect = CountVectorizer(ngram_range=ngram)
      pipe = make_pipeline(vect, tfidf)
      pipe.fit_transform(X_train)
      pipe.transform(X_val)
      logistic_model = LogisticRegression(C=C, class_weight='balanced', multi_class='ovr')
      full_pipeline = make_pipeline(pipe, logistic_model)
      full_pipeline.fit(X_train, y_train)
      y_pred_class = full_pipeline.predict(X_val)
      score = metrics.accuracy_score(y_val, y_pred_class)
      if score > best_score:
        best_ngram = ngram
        best_C = C
        best_score = score

In [23]:
print(best_ngram)
print(best_C)
print(best_score)

(1, 2)
70
0.7957166392092258


**Huấn luyện theo mô hình tốt nhất vừa tìm được**

In [24]:
logistic_model = LogisticRegression(C=best_C, class_weight='balanced', multi_class='ovr')
vect = CountVectorizer(ngram_range=best_ngram)
pipe = make_pipeline(vect, tfidf)
pipe.fit_transform(X_train, y_train)
pipe.transform(X_val)
pipe.transform(X_test)
full_pipeline = make_pipeline(pipe, logistic_model)

**Thực hiện dự đoán kết quả của tập Test**

In [25]:
full_pipeline.fit(X_train, y_train)
y_pred_class = full_pipeline.predict(X_test)
print('Accuracy: ')
print(metrics.accuracy_score(y_test, y_pred_class))


Accuracy: 
0.8031674208144797


**Dump models**

In [26]:
joblib.dump(full_pipeline, "pipeline_logreg.joblib")

['pipeline_logreg.joblib']

# **Huấn luyện theo mô hình Support Vector Machine**

**Thử nghiệm chọn ra mô hình tốt nhất**

In [27]:
svm_C = [1, 10, 20, 30, 40, 50, 60, 70, 80]
svm_ngram = [(1,1), (1,2), (1,3), (2,2), (2,3)]

best_C = 0
best_ngram = (0,0)
best_score = 0.0

for C in svm_C:
    for ngram in svm_ngram:
      vect = CountVectorizer(ngram_range=ngram)
      pipe = make_pipeline(vect, tfidf)
      pipe.fit_transform(X_train)
      pipe.transform(X_val)
      svm_model = svm.SVC(C=C, kernel='linear', degree=3, gamma='auto')
      full_pipeline = make_pipeline(pipe, svm_model)
      full_pipeline.fit(X_train, y_train)
      y_pred_class = full_pipeline.predict(X_val)
      score = metrics.accuracy_score(y_val, y_pred_class)
      if score > best_score:
        best_ngram = ngram
        best_C = C
        best_score = score

In [28]:
print(best_ngram)
print(best_C)
print(best_score)

(1, 2)
10
0.7891268533772653


**Huấn luyện theo mô hình tốt nhất vừa tìm được**

In [29]:
svm_model = svm.SVC(C=best_C, kernel='linear', degree=3, gamma='auto', probability=True)
vect = CountVectorizer(ngram_range=best_ngram)
pipe = make_pipeline(vect, tfidf)
pipe.fit_transform(X_train, y_train)
pipe.transform(X_val)
pipe.transform(X_test)
full_pipeline = make_pipeline(pipe, svm_model)

**Thực hiện dự đoán kết quả của tập Test**

In [30]:
full_pipeline.fit(X_train, y_train)
y_pred_class = full_pipeline.predict(X_test)
print('Accuracy: ')
print(metrics.accuracy_score(y_test, y_pred_class))


Accuracy: 
0.7941176470588235


In [31]:
full_pipeline.steps[1][1]

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

**Dump models**

In [32]:
joblib.dump(full_pipeline, "pipeline_svm.joblib")

['pipeline_svm.joblib']

# **Huấn luyện theo mô hình k-nearest neighbor**

**Thử nghiệm chọn ra mô hình tốt nhất**

In [33]:
knn_neighbors = list(range(1,10))
knn_ngram = [(1,1), (1,2), (1,3), (2,2), (2,3)]

best_n = 0
best_ngram = (0,0)
best_score = 0.0

for n_neighbors in knn_neighbors:
    for ngram in li_ngram:
      vect = CountVectorizer(ngram_range=ngram)
      pipe = make_pipeline(vect, tfidf)
      pipe.fit_transform(X_train)
      pipe.transform(X_val)
      knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)
      full_pipeline = make_pipeline(pipe, knn_model)
      full_pipeline.fit(X_train, y_train)
      y_pred_class = full_pipeline.predict(X_val)
      score = metrics.accuracy_score(y_val, y_pred_class)
      if score > best_score:
        best_ngram = ngram
        best_n = n_neighbors
        best_score = score

In [34]:
print(best_ngram)
print(best_n)
print(best_score)

(1, 2)
9
0.6738056013179572


**Huấn luyện theo mô hình tốt nhất vừa tìm được**

In [35]:
knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)
vect = CountVectorizer(ngram_range=best_ngram)
pipe = make_pipeline(vect, tfidf)
pipe.fit_transform(X_train, y_train)
pipe.transform(X_val)
pipe.transform(X_test)
full_pipeline = make_pipeline(pipe, knn_model)

**Thực hiện dự đoán kết quả của tập Test**

In [36]:
full_pipeline.fit(X_train, y_train)
y_pred_class = full_pipeline.predict(X_test)
print('Accuracy: ')
print(metrics.accuracy_score(y_test, y_pred_class))


Accuracy: 
0.6561085972850679


**Dump models**

In [37]:
joblib.dump(full_pipeline, "pipeline_knn.joblib")

['pipeline_knn.joblib']

# **Thử nghiệm các mô hình**
Dự đoán loại câu hỏi:

**Nhập câu hỏi cần dự đoán ở đây**

In [38]:
x = 'California là tiểu bang của nước nào?'

In [39]:
x = [x]

In [40]:
import pandas as pd 
import re
import string
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline  import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import metrics

import pickle
import joblib

In [41]:
models = []
pipe_nb = joblib.load("pipeline_nb.joblib")
pipe_logreg = joblib.load("pipeline_logreg.joblib")
pipe_svm = joblib.load("pipeline_svm.joblib")
pipe_knn = joblib.load("pipeline_knn.joblib")

models.extend([pipe_nb, pipe_logreg, pipe_svm, pipe_knn])

In [53]:
for model in models:
  print(f'Kết quả dự đoán của mô hình {model.steps[1][0]} là: {model.predict(x)[0]} với độ tin cậy {max(model.predict_proba(x)[0])*100}%')

Kết quả dự đoán của mô hình multinomialnb là: LOC.country với độ tin cậy 16.69377866422538%
Kết quả dự đoán của mô hình logisticregression là: LOC.country với độ tin cậy 86.5280556362957%
Kết quả dự đoán của mô hình svc là: LOC.country với độ tin cậy 91.33346836267015%
Kết quả dự đoán của mô hình kneighborsclassifier là: LOC.country với độ tin cậy 33.33333333333333%
